In [1]:
import os
import glob
import mne
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
from utils import (get_parent_dir, extract_lab_sec, loc_of_sep_lines, plot_seizure_horizontal, plot_seizure_vertical,
                                        raw_to_df, extract_groups, extract_ordered_groups, save_plotly_to_html,
                                        shrink_df_to_tmax, create_results_folders, save_fig_to_disc, plot_interactive_subplot_with_table)

graph_sep_line_width = 5
plot_tmin = 5
plot_tmax = 60

In [21]:
def extract_parameters_from_raw(raw=None):
    highp = raw.info["highpass"]
    lowp = raw.info["lowpass"]
    sfreq = raw.info["sfreq"]
    aq = raw.info["meas_date"]
    channels = raw.info["ch_names"]
    nr_channels = raw.info["nchan"]
    return highp, lowp, sfreq, aq, channels, nr_channels

def raw_to_df(raw, edf=None):
    df = pd.DataFrame(raw.annotations)
    to_drop = ["duration"]
    df = df.drop(to_drop, axis=1)
    if "Beginn" in str(df["description"]):
        samp_beginn = df[df["description"].str.contains("Beginn")]["onset"]
        onset = samp_beginn
        if isinstance(samp_beginn, pd.core.series.Series):
            print(f"There are multiple markers for seizure onset in this file --> taking first one.")
            samp_beginn = samp_beginn.iloc[0]
            onset = samp_beginn
    else:
        print("Error: No marker containing \"Beginn\" found, cannot determine seizure onset for file: ", edf)
        print("Setting seizure onset to the beginning of the file")
        samp_beginn = 0
        onset = "No seizure onset was marked"
    df["time_from_onset"] = df["onset"] - float(samp_beginn)
    df = df.drop(["orig_time"], axis=1)
    return df, onset

In [40]:
edfs = glob.glob("../data/*.edf")
e = edfs[0]
e

'../data/AC_anfall2.edf'

In [42]:
raw = mne.io.read_raw(e)
df, onset = raw_to_df(raw, e)
e_events, s_events, t_events = extract_groups(df, e)
e_events

Extracting EDF parameters from /Users/idrael/git/VEEG_Event_Processor/data/AC_anfall2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Error: No marker containing "Beginn" found, cannot determine seizure onset for file:  ../data/AC_anfall2.edf
Setting seizure onset to the beginning of the file


,onset,description,time_from_onset


In [44]:
def plot_eventcounts(df=None, eeg=None, semio=None, source=None):
    fig, ax = plt.subplots(figsize=(15,26))
    plt.suptitle(str(source) + " - Event counts")
    #EEG
    ax1 = fig.add_subplot(2, 1, 1)
    ax1.set_title("EEG Events")
    if len(eeg['description'].value_counts()) > 0:
        sns.countplot(y="description", data=eeg, orient="h", ax=ax1, order = eeg['description'].value_counts().index)
    else:
        #plt.plot(x=1, y=1)
        ax1.set_title("No EEG events found in file...")
    #Semiology
    ax2 = fig.add_subplot(2, 1, 2)
    ax2.set_title("Semiology Events")
    if [semio['description'].value_counts()] != []:
        sns.countplot(y="description", data=semio, orient="h", ax=ax2, order = semio['description'].value_counts().index)
    else:
        plt.plot(x=1, y=1)
        ax2.set_title("No Semiology events found in file...")
    ax.set_yticks([])
    ax.set_axis_off()
    fig.subplots_adjust(hspace=0.2)
    return fig

fig = plot_eventcounts(df=df, eeg=e_events, semio=s_events, source=e)

TypeError: plot got an unexpected keyword argument 'x'